<a href="https://colab.research.google.com/github/ten-jampa/LLM_grind/blob/main/course/en/chapter3/section7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Pytorch & other libraries
%pip install -qqq torch torchvision setuptools scikit-learn

# Install Hugging Face libraries
%pip install  --upgrade datasets -qqq accelerate hf-transfer transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not

In [11]:
from datasets import load_dataset, DatasetDict

# Dataset id from huggingface.co/dataset
dataset_id = "burtenshaw/PleIAs_common_corpus_code_classification" #code corpus labeled with what programming language it is coded in

# Load raw dataset
dataset = load_dataset(dataset_id)

# Sample the dataset
sample_size = 2000 # You can adjust this number
dataset = DatasetDict({
    'train': dataset['train'].select(range(min(sample_size, len(dataset['train'])))),
    'test': dataset['test'].select(range(min(int(sample_size * 0.1), len(dataset['test'])))) # Sample test set as well
})

In [12]:
print(len(dataset["train"]))
print(dataset["train"][0]) # key of 'text' and 'label'

2000
{'text': '/*\n * Copyright (c) 2000 Kungliga Tekniska Högskolan\n * (Royal Institute of Technology, Stockholm, Sweden).\n * All rights reserved.\n *\n * Redistribution and use in source and binary forms, with or without\n * modification, are permitted provided that the following conditions\n * are met:\n *\n * 1. Redistributions of source code must retain the above copyright\n *    notice, this list of conditions and the following disclaimer.\n *\n * 2. Redistributions in binary form must reproduce the above copyright\n *    notice, this list of conditions and the following disclaimer in the\n *    documentation and/or other materials provided with the distribution.\n *\n * 3. Neither the name of the Institute nor the names of its contributors\n *    may be used to endorse or promote products derived from this software\n *    without specific prior written permission.\n *\n * THIS SOFTWARE IS PROVIDED BY THE INSTITUTE AND CONTRIBUTORS ``AS IS\'\' AND\n * ANY EXPRESS OR IMPLIED WAR

In [13]:
# prompt: transform the datasets['train'] into a panda datafram

import pandas as pd

train_df = pd.DataFrame(dataset['train'])
train_df.head()

,text,labels
0,/*\n * Copyright (c) 2000 Kungliga Tekniska Hö...,C
1,#ifndef STATIC_GRAPH_HPP\n#define STATIC_GRAPH...,C++
2,from BDall import *\nfrom MetodosBD import bas...,Python
3,require 'spec_helper'\ndescribe 'apt::hold' do...,Ruby
4,<?php if ( ! defined('BASEPATH')) exit('No di...,PHP


In [37]:
from transformers import AutoTokenizer
from datasets import DatasetDict, Value

# Model id to load the tokenizer
model_id = "answerdotai/ModernBERT-base" # BERT is an encoder only model

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Tokenize helper function
def tokenize(batch):
    # Map string labels to integer IDs and ensure the type is long
    batch['labels'] = [int(label2id[label]) for label in batch['labels']]
    return tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt")

# Sample the dataset
sample_size = 10000 # You can adjust this number
sampled_dataset = DatasetDict({
    'train': dataset['train'].select(range(min(sample_size, len(dataset['train'])))),
    'test': dataset['test'].select(range(min(int(sample_size * 0.1), len(dataset['test'])))) # Sample test set as well
})

# Tokenize sampled dataset
# Keep the 'labels' column by explicitly listing the columns to remove
tokenized_dataset = sampled_dataset.map(tokenize, batched=True, remove_columns=["text"])

# Set the labels column to ClassLabel with the correct number of classes
tokenized_dataset = tokenized_dataset.cast_column('labels', Value('int64'))


tokenized_dataset["train"].features.keys()
# dict_keys(['labels', 'input_ids', 'attention_mask'])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

KeyError: 'Eagle'

In [15]:
tokenized_train_df = pd.DataFrame(tokenized_dataset['train'])
tokenized_train_df.head()

,labels,input_ids,attention_mask
0,C,"[50281, 3122, 187, 475, 8283, 313, 68, 10, 530...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,C++,"[50281, 4, 17331, 12229, 1864, 64, 6217, 42363...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,Python,"[50281, 4064, 21102, 455, 1395, 475, 187, 4064...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Ruby,"[50281, 15684, 686, 4157, 64, 28316, 8, 187, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,PHP,"[50281, 14277, 5581, 50276, 338, 313, 2195, 29...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [38]:
from transformers import AutoModelForSequenceClassification

# Model id to load the tokenizer
model_id = "answerdotai/ModernBERT-base"

# Prepare model labels - useful for inference
# Use the full dataset to get all unique labels
labels = list(set(dataset["train"]["labels"] + dataset["test"]["labels"])) # get all unique labels from both train and test splits before sampling
num_labels = len(labels) # how many labels are there
label2id, id2label = dict(), dict() #creating a 2 way mapping from id to label and label to id
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [21]:
print(label2id['TSV'])
print(id2label['8'])

8
TSV


In [22]:
# Download the model from huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label,
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
import numpy as np
from sklearn.metrics import f1_score

# Metric helper method
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    score = f1_score(
            labels, predictions, labels=labels, pos_label=1, average="weighted"
        )
    return {"f1": float(score) if score == 1 else score}


In [32]:
from huggingface_hub import HfFolder
from transformers import Trainer, TrainingArguments

# Define training args
training_args = TrainingArguments(
    output_dir= "ModernBERT-code-classifier",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=32,
    learning_rate=5e-5,
    num_train_epochs=5,
    bf16=True, # bfloat16 training
    optim="adamw_torch_fused", # improved optimizer
    # logging & evaluation strategies
    logging_strategy="steps",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    # push to hub parameters
    push_to_hub=True,
    hub_strategy="every_save",
    hub_token=HfFolder.get_token(),
    report_to="wandb",
    gradient_checkpointing=True
)



# Overfitting

In [26]:
# prompt: login to hugging face

from huggingface_hub import notebook_login

notebook_login()

In [35]:
print(tokenized_dataset['train'])

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 2000
})


In [34]:
limited_dataset = tokenized_dataset["train"].select(range(100))
print(limited_dataset)
# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=limited_dataset,
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,

)
trainer.train()

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 100
})


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.

In [ ]:
# clear memory

import torch
torch.cuda.empty_cache()

del trainer
del model
del limited_dataset

# Underfitting

In [ ]:
# define a low learning rate
training_args.learning_rate = 1e-7

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=limited_dataset,
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)
trainer.train()

In [ ]:
# clear memory

import torch
torch.cuda.empty_cache()

del trainer
del model

# Just right! 🥣

In [ ]:
# define a valid learning rate
training_args.learning_rate = 5e-5

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=limited_dataset,
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)
trainer.train()

# Inference

In [ ]:
from transformers import pipeline

# load model from huggingface.co/models using our repository id
classifier = pipeline(
    task="text-classification",
    model="argilla/ModernBERT-domain-classifier",
    device=0,
)

sample = """def add_numbers(a, b):
    return a + b"""

classifier(sample)


In [29]:
import torch

torch.cuda.empty_cache()
print("GPU memory cleared.")

GPU memory cleared.


In [36]:
print(tokenized_dataset["train"].features['labels'].dtype)

string
